# 13-9. 프로젝트 - Movielens 영화 추천 실습

## 0. 환경설정 및 필요한 라이브러리 Import

In [1]:
# ! mkdir -p ~/aiffel/recommendata_iu/data/ml-1m
# ! ln -s ~/data/ml-1m/* ~/aiffel/recommendata_iu/data/ml-1m

In [2]:
import os
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import scipy
import implicit

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


## 1. 데이터 준비와 전처리

### 1-1. 데이터 준비

In [3]:
# Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## 1-2. 데이터 전처리

In [4]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [5]:
# 평점을 시청횟수로 해석할것이므로 ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
# movies와 ratings를 합쳐준다.
data= pd.merge(movies, ratings)

In [8]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = [ 'user_id','title', 'counts']
data = data[using_cols]
data.head(10)

,user_id,title,counts
0,1,Toy Story (1995),5
1,6,Toy Story (1995),4
2,8,Toy Story (1995),4
3,9,Toy Story (1995),5
4,10,Toy Story (1995),5
5,18,Toy Story (1995),4
6,19,Toy Story (1995),5
7,21,Toy Story (1995),3
8,23,Toy Story (1995),4
9,26,Toy Story (1995),3


In [9]:
# 검색 및 훈련을 위해서 제목의 대문자를 모두 소문자로 바꿈
data['title'] = data['title'].str.lower() 

## 2. 데이터 분석

### 2-1. ratings에 있는 유니크한 영화 개수

In [10]:
data['title'].nunique()

3628

### 2-2. ratings에 있는 유니크한 사용자 수

In [11]:
data['user_id'].nunique()

6039

### 2-3. 가장 인기 있는 영화 30개(인기순)

In [12]:
movies_count = data.groupby('title')['user_id'].count()
movies_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

## 3. 내가 선호하는 영화를 5가지 골라서 data에 추가

In [13]:
# 내가 좋아하시는 영화의 데이터로 바꿔서 추가
my_favorite = ['saving private ryan (1998)', 'sixth sense, the (1999)', 'godfather, the (1972)',
               'gladiator (2000)', 'die hard (1988)']

# 'YJKim'이라는 user_id가 위 영화의 평점을 5점씩 주었다고 가정
my_playlist = pd.DataFrame({'user_id': ['YJKim']*5, 'title': my_favorite, 'counts':[5]*5})

# user_id에 'YJKim'이라는 데이터가 없다면 위에 임의로 만든 my_favorite 데이터를 추가해 줌 
if not data.isin({'user_id':['YJKim']})['user_id'].any():  
    data = data.append(my_playlist)   
    
data.tail(20) 

,user_id,title,counts
836463,5074,"contender, the (2000)",4
836464,5087,"contender, the (2000)",4
836465,5100,"contender, the (2000)",4
836466,5205,"contender, the (2000)",4
836467,5304,"contender, the (2000)",4
836468,5333,"contender, the (2000)",4
836469,5359,"contender, the (2000)",5
836470,5405,"contender, the (2000)",4
836471,5475,"contender, the (2000)",5
836472,5602,"contender, the (2000)",3


In [14]:
# 고유한 유저, 타이틀을 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 타이틀 indexing 하는 코드 (idx는 index의 약자)
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [15]:
# 인덱싱이 잘 되었는지 확인 
print(user_to_idx['YJKim'])  # 6040명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 함 
print(title_to_idx['saving private ryan (1998)'])
print(title_to_idx['sixth sense, the (1999)'])
print(title_to_idx['godfather, the (1972)'])
print(title_to_idx['gladiator (2000)'])
print(title_to_idx['die hard (1988)'])

6039
1810
2507
785
3270
949


#### 내가 점수를 부여한 영화는 인덱스가 3628보다 작아 모두 기존 영화목록에 있다.

In [16]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드 (새로 추가한 유저(YJKim)를 indexing)

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구함 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체 
else:
    print('user_id column indexing Fail!!')

# temp_title_data을 통해 title 컬럼도 동일한 방식으로 인덱싱 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,counts
0,0,0,5
1,1,0,4
2,2,0,4
3,3,0,5
4,4,0,5
...,...,...,...
0,6039,1810,5
1,6039,2507,5
2,6039,785,5
3,6039,3270,5


## 4. CSR(Compressed Sparse Row) Matrix 작성

In [17]:
# 유저수와 영화수 확인
data['user_id'].nunique(), data['title'].nunique()

(6040, 3628)

#### 총 6,040명의 유저와 3,628편의 영화가 있다.

In [18]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

#### 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들어졌다.

## 5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련

In [19]:
from implicit.als import AlternatingLeastSquares

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [20]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=200, regularization=0.02, use_gpu=False, iterations=20, dtype=np.float32)

In [21]:
# als 모델은 input으로 (item X user) 꼴의 matrix를 받기 때문에 Transpose해줌
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [22]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

In [23]:
# YJKim 벡터와 gladiator (2000)의 벡터를 어떻게 만들고 있는지
YJKim, gladiator = user_to_idx['YJKim'], title_to_idx['gladiator (2000)']
YJKim_vector, gladiator_vector = als_model.user_factors[YJKim], als_model.item_factors[gladiator]

In [24]:
YJKim_vector

array([ 2.32476577e-01,  4.00793217e-02,  1.47631481e-01, -4.63902146e-01,
       -3.69511455e-01,  4.04684879e-02, -1.76119953e-02,  1.54797137e-01,
       -4.86532480e-01,  2.13015676e-01,  3.87391686e-01, -5.46569526e-01,
        2.27004543e-01,  1.05152406e-01,  6.70004189e-02,  1.03853345e-02,
       -1.45547166e-01, -1.54474467e-01,  3.55242074e-01, -2.31961384e-01,
       -1.21750832e-01, -2.34763145e-01,  9.20907199e-01, -3.06998789e-01,
       -3.00520808e-01,  1.58820003e-01, -5.19637344e-03,  1.00380249e-01,
        1.52297392e-01,  5.46943128e-01, -1.12791806e-01, -2.33909994e-01,
        4.98952746e-01, -2.81647354e-01, -1.95313498e-01, -1.68137178e-02,
        1.00516665e+00,  2.28927314e-01, -3.07395279e-01,  3.34239811e-01,
        1.75444275e-01, -1.11226238e-01,  1.37898728e-01,  2.00657308e-01,
       -6.18456721e-01, -4.44711186e-02,  3.67960297e-02,  4.47388172e-01,
       -8.20972472e-02,  7.88678885e-01,  5.63937910e-02,  6.21792441e-03,
       -1.06444836e-01, -

In [25]:
gladiator_vector

array([ 0.03654819, -0.00080493,  0.03310014, -0.00912815, -0.03599286,
        0.01224669, -0.02878761,  0.01536378, -0.0131797 ,  0.02819902,
        0.04175749, -0.02452933, -0.01116403, -0.01710727,  0.0280606 ,
        0.02228355, -0.00728517, -0.02022888,  0.01792426, -0.02952919,
        0.00526002,  0.00651958,  0.02043268,  0.00080293, -0.01659622,
        0.06473186,  0.00386107,  0.02133526,  0.02606721,  0.04340047,
        0.01704863, -0.00721647,  0.0032033 ,  0.00198149,  0.01459343,
        0.00089862,  0.01639691, -0.00511509,  0.01205988,  0.07509709,
        0.01647216,  0.04523923,  0.00911622,  0.04528669, -0.02864759,
        0.02228835,  0.00595182,  0.05867262,  0.01112861,  0.03778834,
       -0.06642418, -0.00903842, -0.03015688,  0.00225241,  0.01436729,
        0.02551956,  0.00203922, -0.01672438, -0.00222845, -0.02745043,
        0.02918672, -0.02799186, -0.01522177, -0.02719835,  0.01996326,
       -0.0531535 ,  0.00971145,  0.00963425,  0.00786048,  0.01

## 6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악

In [26]:
# YJKim과 gladiator (2000)를 내적하는 코드
np.dot(YJKim_vector, gladiator_vector)

0.8005463

In [27]:
# 모델이 내가 선호하는 godfather, the (1972)에 대한 선호도 예측
godfather = title_to_idx['godfather, the (1972)']
godfather_vector = als_model.item_factors[godfather]
np.dot(YJKim_vector, godfather_vector)

0.7771147

In [28]:
# 모델이 그 외의영화 forrest gump (1994)에 대한 선호도 예측
forrest_gump = title_to_idx['forrest gump (1994)']
forrest_gump_vector = als_model.item_factors[forrest_gump]
np.dot(YJKim_vector, forrest_gump_vector)

0.027053438

#### 내가 선호하는 godfather, the (1972)에 대해서는 0.777의 매우 높은 선호도를 나타낸 반면 장르가 전혀 다른
#### forrest gump (1994)에 대해서는 0.027의 매우 낮은 선호도를 나타내고있어 훈련이 잘 되었다고 판단된다.

## 7. 내가 좋아하는 영화와 비슷한 영화를 추천받기

In [29]:
#  반복해서 확인하기 위해 위의 코드를 함수로 작성

idx_to_title = {v:k for k,v in title_to_idx.items()}
def get_similar_title(movie_name: str):
    movie_id = title_to_idx[movie_name]
    similar_title = als_model.similar_items(movie_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

In [30]:
get_similar_title('die hard (1988)')

['die hard (1988)',
 'terminator, the (1984)',
 'indiana jones and the last crusade (1989)',
 'untouchables, the (1987)',
 'fugitive, the (1993)',
 'die hard 2 (1990)',
 'raiders of the lost ark (1981)',
 'rocky (1976)',
 'predator (1987)',
 'aliens (1986)']

In [31]:
get_similar_title('sixth sense, the (1999)')

['sixth sense, the (1999)',
 'usual suspects, the (1995)',
 'end of days (1999)',
 'seven (se7en) (1995)',
 'ghostbusters (1984)',
 'silence of the lambs, the (1991)',
 'bone collector, the (1999)',
 'eyes wide shut (1999)',
 'superstar (1999)',
 'fugitive, the (1993)']

#### 내가 점수를 부여했던 영화를 입력하고 비슷한 영화를 추천받아본 결과 매우 유사한 장르의 영화들이 추천되었다.

## 8. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [32]:
user = user_to_idx['YJKim']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
[idx_to_title[i[0]] for i in title_recommended]

['godfather: part ii, the (1974)',
 'fugitive, the (1993)',
 'raiders of the lost ark (1981)',
 'braveheart (1995)',
 'rocky (1976)',
 'indiana jones and the last crusade (1989)',
 'glory (1989)',
 "schindler's list (1993)",
 'terminator, the (1984)',
 'mission: impossible 2 (2000)',
 'jaws (1975)',
 'lethal weapon (1987)',
 'matrix, the (1999)',
 'perfect storm, the (2000)',
 'patriot, the (2000)',
 'terminator 2: judgment day (1991)',
 'usual suspects, the (1995)',
 'die hard 2 (1990)',
 'full metal jacket (1987)',
 'untouchables, the (1987)']

#### 모두 내가 좋아하는 장르의 영화들이 추천되었다.

In [33]:
# 위에 godfather: part ii, the (1974)를 추천받은데 대한 내가 평가한 영화들의 기여도를 확인
godfather = title_to_idx['godfather: part ii, the (1974)']
explain = als_model.explain(user, csr_data, itemid=godfather)

In [34]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('godfather, the (1972)', 0.6049795472635341),
 ('die hard (1988)', 0.023904302742023938),
 ('saving private ryan (1998)', 0.007321160293261753),
 ('sixth sense, the (1999)', -0.003208932446517628),
 ('gladiator (2000)', -0.016921928212423006)]

#### 첫번째로 추천받은 godfather: part ii, the (1974)에대한 기여도를 분석한 결과 시리즈물인 godfather, the (1972)가
#### 0.6으로 가장 많이 기여하였고 장르가 조금 다른 gladiator (2000)는 (-)의 기여도가 확인되어 모델이 잘 훈련
#### 되었다고 판단된다.

## 회고
### 이번 프로젝트에서 어려웠던 점

CSR Matrix 작성시 "ValueError: row index exceeds matrix dimensions" 에러가 발생하여 원인을 찾지 못해 고생했다.
새로 추가된 유저를 indexing하면서 원인이 제거된것으로 생각된다.

### 프로젝트를 진행하면서 알아낸 점 혹은 아직 모호한 점

모델훈련을 반복하면서 좋은 결과값이 나와 모든 셀을 다시 실행시키면 값이 또 바뀐다. iteration이 누적될 가능성이 있지 않을까?

### 루브릭 평가 지표를 맞추기 위해 시도한 것들

Hyperparameter인 factors, regularization, iterations 등을 다양하게 바꾸어 보았다.

### 만약에 루브릭 평가 관련 지표를 달성 하지 못했을 때, 이유에 관한 추정

충분히 달성했다고 생각된다.

### 자기 다짐

추천시스템은 인터넷을 사용하면서 늘 접하는 분야로 평소 궁금했는데 관심갖고 공부하여 기회가 되었을 때 활용할 수 있도록 하겠다.